In [ ]:
### Load XH

In [1]:
import glob,numpy as np,sys,os,pickle
iset = 6
dec_fls = np.sort(glob.glob(rf'Z:\MERFISHp\12_16_2025_BigSlideTest\decoded_files_th3_old\Decoded*zscan{iset}_*'))
len(dec_fls)

290

In [2]:
dic_pos_fl = r'Z:/MERFISHp/12_16_2025_BigSlideTest/dic_pos_pixf.pkl'
dic_pos = pickle.load(open(dic_pos_fl,'rb'))

fovs = np.sort([fov for fov in dic_pos if f'zscan{iset}_' in fov])
len(fovs)

836

In [19]:
import os
import numpy as np
from scipy.spatial import KDTree
from scipy import ndimage
class dummy():
    def __init__(self):
        pass
def return_X_spots(fl_dec, th = -1.5):
    """
    Load decoded spots, compute scoreA, threshold, drift-correct (H1→H5),
    assign to segmentation, and save BOTH:
      - <fov>--spots.npz  (per-spot table)
      - <fov>--matrix.npz (cell × gene counts)
    """
    

    # --- figure out set, and use the REAL xfits folder
    
    dec = dummy()
    # --- load decoded (will raise if file is missing/corrupt)
    dec.ncols = 3
    fov = os.path.basename(fl_dec).split('--')[-1].replace('.npz','')
    iset = int(fov.split('zscan')[-1].split('_')[0])
    set_ = f'_set{iset}'
    dec.set_ = set_
    dec.fov = fov
    dec.decoded_fl = fl_dec
    dec.Xkp=[]
    dec.genes=[]
    if not os.path.exists(dec.decoded_fl):
        return dec
    try:
        dec.XH_pruned = np.load(dec.decoded_fl)['XH_pruned']
        dec.icodesN = np.load(dec.decoded_fl)['icodesN']
        dec.gns_names = np.load(dec.decoded_fl)['gns_names']
        dec.dist_best = np.load(dec.decoded_fl)['dist_best']
        dec.scoreA = np.load(dec.decoded_fl)['scoreA']
    except:
        return dec


    # --- threshold
    
    #dec.th = -0.4
    if type(th) is np.ndarray:
        Xs = np.nanmean(dec.XH_pruned[:,:,:3],axis=1)
        rescT = 10
        x,y = np.round(Xs[:,1]/rescT).astype(int),np.round(Xs[:,2]/rescT).astype(int)
        keep_th = (x>=0)&(x<th.shape[0])&(y>=0)&(y<th.shape[1])
        dec.th = np.zeros(len(Xs))
        dec.th[keep_th]=th[x[keep_th],y[keep_th]]
    else:
        dec.th=th
    keep            = dec.scoreA > dec.th
    icodesN         = dec.icodesN[keep]
    XH_pruned_kept  = dec.XH_pruned[keep]           # (Nkeep, nbits, features)

    XHT = np.nanmean(XH_pruned_kept,axis=1)
    ifov = int(fov.split('_')[-1])
    XHT[:,-1]=ifov
    XHT[:,-2]=dec.scoreA[keep]
    Xkp = XHT[:,[0,1,2,-2,-1]]
    genes = dec.gns_names[icodesN]
    dec.Xkp=Xkp
    dec.genes=genes
    return dec
from tqdm import tqdm

In [20]:
iset=5
import glob,numpy as np,sys,os,pickle
dec_fls = np.sort(glob.glob(rf'Z:\MERFISHp\12_16_2025_BigSlideTest\decoded_files_th3\Decoded*zscan{iset}_*'))
def get_dec_fl_alt(dec_fl): return dec_fl.replace('decoded_files_th3','decoded_files_th3_old')
dec_fls_old_ = np.sort(glob.glob(rf'Z:\MERFISHp\12_16_2025_BigSlideTest\decoded_files_th3_old\Decoded*zscan{iset}_*'))
dec_fls_old = [get_dec_fl_alt(dec_fl) for dec_fl in dec_fls if os.path.exists(get_dec_fl_alt(dec_fl))]
#len(dec_fls)==len(dec_fls_old)
all_fovs = [fov for fov in dic_pos if f'zscan{iset}_' in fov]
len(dec_fls_old_),len(dec_fls),len(all_fovs)

(966, 966, 966)

In [22]:
def get_fov(dec_fl): return dec_fl.split('--')[-1].split('.')[0]
completed_fovs = [get_fov(fl) for fl in dec_fls]
np.setdiff1d(all_fovs,completed_fovs)

array([], dtype='<U16')

In [23]:
import glob,numpy as np,sys,os,pickle
dec_fls = np.sort(glob.glob(rf'Z:\MERFISHp\12_16_2025_BigSlideTest\decoded_files_th3\Decoded*zscan{iset}_*'))
XH,genes = [],[]
for fov in tqdm(all_fovs):
    fl_dec1 = rf'Z:\MERFISHp\12_16_2025_BigSlideTest\decoded_files_th3\DecodedNew--{fov}.npz'
    fl_dec2 = rf'Z:\MERFISHp\12_16_2025_BigSlideTest\decoded_files_th3_old\DecodedNew--{fov}.npz'
    dec1 = return_X_spots(fl_dec1, th = -1.5)
    dec2 = return_X_spots(fl_dec2, th = -1.5)
    dic1 = dict(zip(*np.unique(dec1.genes,return_counts=True)))
    dic2 = dict(zip(*np.unique(dec2.genes,return_counts=True)))
    all_genes = np.unique(list(dic1.keys())+list(dic2.keys()))
    fr = np.mean([dic2.get(gn,0)>=dic1.get(gn,0)for gn in all_genes if 'blank' not in gn])
    if fr>0.5:
        dec=dec2
    else:
        dec=dec1
    XH_,genes_ = dec.Xkp,dec.genes
    XH.append(XH_)
    genes.append(genes_)
XH=np.concatenate(XH)
genes=np.concatenate(genes)



dic_pos_fl = r'Z:/MERFISHp/12_16_2025_BigSlideTest/dic_pos_pixf.pkl'
dic_pos = pickle.load(open(dic_pos_fl,'rb'))

ifovs_XF = XH[:,-1].astype(int)

dic_pos_ = {int(fov.split('_')[-1]):dic_pos[fov]*0.108333 for fov in dic_pos if f'zscan{iset}_' in fov}
pos_fovs = np.array([dic_pos_.get(ifov) for ifov in np.arange(1+np.max(ifovs_XF))])

ifovs_XF = XH[:,-1].astype(int)
X = XH[:,1:3]*0.108333+pos_fovs[ifovs_XF]#*[-1,-1]
pos_fovs_cm = pos_fovs+2800*0.108333/2

dd,ii = KDTree(pos_fovs_cm).query(X,workers=10,p=np.inf)
kpU = ii==ifovs_XF#[kp]
X2d = X[kpU]

pos_fovs_pix = pos_fovs_cm/0.108333-2800/2
Pmin = np.min(pos_fovs_pix,axis=0)
X2d_pix = X2d/0.108333-Pmin
pos_fovs_pix = pos_fovs_pix-Pmin
XF = XH[kpU]
XF[:,1:3]=X2d_pix
genesf = genes[kpU]

dic_fovs = {int(fov.split('_')[-1]):fov for fov in dic_pos if f'zscan{iset}_' in fov}
fovs = np.array([dic_fovs.get(ifov,'') for ifov in np.arange(1+np.max(ifovs_XF))])

np.savez(rf'Z:\MERFISHp\12_16_2025_BigSlideTest\final_spots\spots_set{iset}_final.npz',
          XF=XF,genes=genesf,pix_size=[0.4,0.108333,0.108333],fov_size=[40,2800,2800],pos_fovs_pix=pos_fovs_pix,fovs=fovs)



100%|██████████████████████████████████████████████████████████████████████████████| 966/966 [4:48:34<00:00, 17.92s/it]


MemoryError: Unable to allocate 87.0 GiB for an array with shape (1796173910,) and data type <U13

In [238]:
# #dec_fl = dec_fls[0]

# import glob,numpy as np,sys,os,pickle
# dec_fls = np.sort(glob.glob(rf'Z:\MERFISHp\12_16_2025_BigSlideTest\decoded_files_th3\Decoded*zscan{iset}_*'))
# XH,genes = [],[]
# for dec_fl in tqdm(dec_fls):
#     XH_,genes_ = return_X_spots(dec_fl, th = -1.5)
#     XH.append(XH_)
#     genes.append(genes_)
# XH=np.concatenate(XH)
# genes=np.concatenate(genes)

100%|████████████████████████████████████████████████████████████████████████████████| 586/586 [38:37<00:00,  3.96s/it]


In [239]:

dic_pos_fl = r'Z:/MERFISHp/12_16_2025_BigSlideTest/dic_pos_pixf.pkl'
dic_pos = pickle.load(open(dic_pos_fl,'rb'))

ifovs_XF = XH[:,-1].astype(int)

dic_pos_ = {int(fov.split('_')[-1]):dic_pos[fov]*0.108333 for fov in dic_pos if f'zscan{iset}_' in fov}
pos_fovs = np.array([dic_pos_.get(ifov) for ifov in np.arange(1+np.max(ifovs_XF))])

ifovs_XF = XH[:,-1].astype(int)
X = XH[:,1:3]*0.108333+pos_fovs[ifovs_XF]#*[-1,-1]
pos_fovs_cm = pos_fovs+2800*0.108333/2

dd,ii = KDTree(pos_fovs_cm).query(X,workers=10,p=np.inf)
kpU = ii==ifovs_XF#[kp]
X2d = X[kpU]

pos_fovs_pix = pos_fovs_cm/0.108333-2800/2
Pmin = np.min(pos_fovs_pix,axis=0)
X2d_pix = X2d/0.108333-Pmin
pos_fovs_pix = pos_fovs_pix-Pmin
XF = XH[kpU]
XF[:,1:3]=X2d_pix
genesf = genes[kpU]
np.savez(rf'Z:\MERFISHp\12_16_2025_BigSlideTest\final_spots\spots_set{iset}.npz',
          XF=XF,genes=genesf,pix_size=[0.4,0.108333,0.108333],fov_size=[40,2800,2800],pos_fovs_pix=pos_fovs_pix,fovs=fovs)

In [ ]:
 'Zfhx4', 'Necab1', 'Col27a1', 'Glis3', 'Npnt', 'Prkd1', 'Stxbp6',
       'Pappa', 'Ntng1', 'Grm8', 'Fign', 'Arhgap25', 'Kcng2', 'Htr1b',
       'Dach1', 'Adra1b', 'Calb1', 'Nwd2', 'Rspo1', 'Grik1', 'Tent5a',
       'Hopx', 'Ccdc3', 'Baiap3', 'Emx2', 'Cbln4', 'Tox3', 'Tbcc', 'Mkx',
       'Cyp26b1', 'Tnc', 'Adgrf5', 'Cd247', 'Ctss', 'Mpped2', 'Cdkn1a',
       'Adra1a', 'Svep1', 'Igf2', 'Gpr83', 'Ecel1', 'Barhl2', 'Sox6',
       'Sema3e', 'Pou3f1', 'Tox', 'Zfpm2', 'Cdh6', 'Ntn1', 'Pde1a',
       'St3gal1', 'Tcf7l2', 'Gfra1', 'Hspg2', 'Slit2', 'Opalin', 'Ighm',
       'Gli3', 'Man1a', 'Bmpr1b', 'Cd34', 'Penk', 'Nrp1', 'Cdh20',
       'Fras1', 'Lmo3', 'Adamts2', 'Synpr', 'Rprm', 'Col5a1', 'Galnt14',
       'Tgfbr2', 'Kctd8', 'Ebf1', 'Foxo1', 'Oprd1', 'Car4', 'Cgnl1',
       'Igfbp6', 'Gda', 'St18', 'Ldb2', 'Drd2', 'Agt', 'Tbx3', 'Otof',
       'C1ql3', 'Hs3st2', 'Ramp1', 'Kcng1', 'Lpar1', 'Rab3b', 'Tmem132c',
       'Sv2c', 'Rorb', 'Rgs6', 'Elfn1', 'Wnt7b', 'Nr2f2', 'Kit', 'Cnih3',
       'Fibcd1', 'Maf', 'Mog', 'Sema5a', 'Tafa2', 'Chrm3', 'Mdga1', 'Ust',
       'Osbpl3', 'Lhfp', 'Igfbp2', 'Syndig1', 'Slc7a10', 'Chn2', 'Syt17',
       'Calb2', 'Fxyd6', 'Nr4a3', 'Pvalb', 'Trhde', 'Lypd1', 'Crym',
       'Sox2', 'Ccn3', 'Nrn1', 'Ptprm', 'Adcy2', 'Gad2', 'St6galnac5',
       'Shisa6', 'Nr4a2', 'Aqp4', 'Sox5', 'Cacng3', 'Ccnd2', 'Nptx2',
       'A830036E02Rik', 'Lamp5', 'Igfbp4', 'Parm1', 'Pdzrn3', 'Zfp521',
       'Hs3st4', 'Zfp536', 'Unc5d', 'Bcl11a', 'Kcnab3', 'Megf11', 'Lrp4',
       'Cxcl14', 'Pcp4l1', 'Cldn5', 'Sorcs3', 'Serpine2', 'Dscaml1',
       'Nos1', 'Fosl2', 'Cnr1', 'Sox10', 'Grm1', 'Galnt18', 'Sulf2',
       'Grik3', 'Slit3', 'Lhx2', 'Reln', 'Syndig1l', 'Slc30a3', 'Tshz2',
       'Whrn', 'Spon1', 'Rgs4', 'Zbtb16', 'Kcnk9', 'Unc13c', 'Slc38a1',
       'Grm3', 'Gja1', 'Nr2f1', 'Zeb2', 'Syt2', 'Nfib', 'Epha4', 'Gpr88',
       'Cbln1', 'Caln1', 'Bcl11b', 'Pou3f3', 'Scn4b', 'Zic1', 'Grin2c',
       'Slc32a1', 'Sv2b', 'Rasgrp1', 'Gfap', 'Slc1a3', 'Nfix', 'Ptk2b',
       'Slc17a7'], dtype='<U13')

In [34]:
'Drd1' in genesf[::1000]

True

In [35]:
import napari
gene = 'Drd1'
V = napari.Viewer()
#for gene in ['Glis3']:#['Gfap', 'Slc1a3','Ptk2b', 'Slc17a7']:
kp = genesf==gene
Xgn = XF[kp][:,1:3]

col = np.random.random(3)
col = [1,1,1]
V.add_points(Xgn,size=5,border_width=0,face_color=list(col)+[0.1],name=gene)
#V.layers[-1].visible=False

<Points layer 'Drd1' at 0x16c81bce020>

In [246]:
XF[kp][304806][-1]

np.float64(155.0)

In [250]:
XF[kp][2375046][-1]

np.float64(229.0)

In [ ]:
FOV155,225

In [242]:
ugns,ctsgns = np.unique(genes[::100],return_counts=True)
ugns[np.argsort(ctsgns)]

array(['blank0036', 'blank0046', 'blank0103', 'blank0068', 'blank0032',
       'Trhr', 'blank0019', 'blank0089', 'blank0048', 'Chst9', 'Foxb1',
       'blank0027', 'blank0112', 'blank0097', 'blank0094', 'Ttc6',
       'blank0023', 'Kcnj5', 'blank0118', 'blank0096', 'blank0099',
       '9330158H04Rik', 'Arhgap15', 'blank0031', 'blank0101', 'Hoxb3',
       'blank0120', 'blank0072', 'Ttc29', 'Drd3', 'blank0100',
       'blank0093', 'blank0102', 'blank0098', 'blank0006', 'blank0042',
       'blank0058', 'blank0074', 'D130009I18Rik', 'blank0107',
       'blank0049', 'blank0018', 'blank0092', 'B230323A14Rik',
       'blank0051', 'blank0111', 'Onecut3', 'blank0026', 'blank0091',
       'Evx2', 'Slc6a4', 'blank0060', 'blank0045', 'blank0121',
       'G630016G05Rik', 'blank0053', 'Apela', 'Hoxb5', 'blank0122',
       'Hpgd', 'blank0090', 'blank0024', 'Vsx2', 'blank0022', 'blank0088',
       'blank0017', 'blank0035', 'blank0005', 'blank0116', 'blank0034',
       'Pax7', 'blank0062', 'blank0108',

In [214]:
iset

4

In [127]:
#XH[kp][kpU][667673][-1]

In [195]:
import napari
gene = 'St6galnac5'
V = napari.Viewer()
#for gene in ['Glis3']:#['Gfap', 'Slc1a3','Ptk2b', 'Slc17a7']:
kp = genes==gene
Xgn = X[kp]
dd,ii = KDTree(pos_fovs_cm).query(Xgn)
kpU = ii==ifovs_XF[kp]
Xgn = Xgn[kpU]
#import matplotlib.pylab as plt
#plt.scatter(Xgn[:,0],Xgn[:,1],s=0.0001)
#plt.axis('equal')
col = np.random.random(3)
col = [1,1,1]
V.add_points(Xgn,size=1,border_width=0,face_color=list(col)+[0.1],name=gene)
#V.layers[-1].visible=False

<Points layer 'St6galnac5' at 0x26487b11060>

In [90]:
len(Xgn)

330614

In [60]:
# import matplotlib.pylab as plt
# plt.scatter(Xgn[:,0],Xgn[:,1],s=0.1,alpha=0.0025)
# plt.axis('equal')

In [23]:
lens = [len(np.load(dec_fl)) for dec_fl in tqdm(dec_fls)]
    

100%|████████████████████████████████████████████████████████████████████████████████| 586/586 [00:10<00:00, 56.69it/s]


In [25]:
np.unique(lens,return_counts=True)

(array([1, 5]), array([  1, 585]))

In [26]:
np.array(dec_fls)[np.array(lens)!=5]

array(['Z:\\MERFISHp\\12_16_2025_BigSlideTest\\decoded_files\\DecodedNew--Conv_zscan4__042.npz'],
      dtype='<U82')